**_♡텍스트 마이닝 사용 설명서♡ : (1) ~ (4)편_**

**(1) 텍스트 분석 기초**   
https://dacon.io/competitions/official/235946/codeshare/5986?page=1&dtype=random   
**(2) 감성 분석**   
https://dacon.io/competitions/official/235946/codeshare/6006?page=1&dtype=random   
**(3) 토픽 모델링**   
https://dacon.io/competitions/official/235946/codeshare/6017?page=1&dtype=random   
**(4) 문서 군집화와 문서 유사도**    
https://dacon.io/competitions/official/235946/codeshare/6031?page=1&dtype=random
   
-----------

# 1. 토픽 모델링(Topic Modeling)이란?
다수의 문서 집합 안에서 같이 사용되는 단어의 집합으로 문서에 내재되어 있는 토픽을 분석하는 기법
- 주로 연구 동향이나 사회적 현상을 파악하기 위한 연구 기법으로 활용   
- 워드클라우드처럼 단순히 단어의 빈도 수를 세는 것에서 넘어 함께 사용되는 단어를 묶어줌으로써 문서의 내용을 구체적으로 예상할 수 있다.
- LDA(Latent Dirichlet Allocation) 알고리즘과 LSA(Latent Semantic Analysis) 알고리즘

# 2. LDA(Latent Dirichlet Allocation)
토픽 모델링에서 가장 빈번하게 사용.   

함께 자주 나타나는 단어의 **토픽**을 찾는 것을 목표로 하는 모델.
- 머신러닝에서의 토픽 : 의미가 있든 없든 PCA나 NMF로 추출한 성분에 가까운 것. 일반적인 주제와는 관련 X

사이킷런은 LDA 기반의 토픽 모델링을 LatentDirichletAllocation 클래스로 제공한다.   
또한, LDA는 Count 기반의 벡터화만 사용한다.

**[LatentDirichletAllocation의 주요 파라미터]**
1) **c_components** : 토픽의 개수 지정   
2) **learning_method** : 'batch' 또는 'online'으로 설정. 디폴트는 online. 데이터 크기가 크면 일반적으로 batch로 설정. online이 batch보다 업데이트 속도는 빠름.

**[CountVectorizer의 주요 파라미터]**
1) **max_df** : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터   
2) **min_df** : 전체 문서에 걸쳐서 너무 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터   
3) **max_features** : 추출하는 피처의 개수를 제한하며 정수로 값을 지정   
4) **stop_words** : "english"로 지정하면 영어의 스롭워드로 지정된 단어는 추출에서 제외   
5) **n_gram_range** : BOW 모델의 단어 순서를 어느 정도 보강하기 위한 n_gram 범위 설정. 튜플 형태로 (범위 최솟값, 범위 최댓값)을 지정.   
6) **analyzer** : 피처 추출을 수행한 단위를 지정. 디폴트는 "word"   
7) **token_pattern** : 토큰화를 수행하는 정규 표현식 패터을 지정. 디폴트는 공백 또는 개행 문자 등으로 구분된 단어 분리자 사이의 2문자 이상 단어.   
8) **tokenizer** : 토큰화를 변도의 커스텀 함수로 이용 시 적용.   

**[LDA에 활용되는 주요 함수]**
1) **LatentDirichletAllocation.fit(df)** : LatentDirichletAllocation 객체는 components_속성값을 갖게 됨.   
2) **.components_** : 개별 토픽별로 각 word 피처가 얼마나 많이 그 토픽에 할당됐는지에 대한 수치   
3) **count_vect.get_feature_names()** : 카운트 벡터화에 사용된 feature 단어 목록을 볼 수 있음

# 3. 20 뉴스 그룹 데이터로 LDA 토픽 모델링 하기

In [1]:
# 기본 라이브러리 설치
import pandas as pd
import numpy as np

In [2]:
# 데이터 불러오기
from sklearn.datasets import fetch_20newsgroups
fetch_20newsgroups

<function sklearn.datasets._twenty_newsgroups.fetch_20newsgroups(*, data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False)>

## 8개 주제 추출 
8개 주제를 추출하여 LDA 기반 토픽 모델링을 적용해 보자!
- 오토바이, 야구, 그래픽스, 윈도우, 중동, 기독교, 전자공학, 의학

LDA 토픽모델링을 위해 categories 파라미터를 통해 **필요한 주제만 필터링하여 추출된 텍스트를 Count 기반으로 백터화 변환**(LDA는 Count 기반의 벡터화만 사용하므로)

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 8개 주제 추출
topics = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x', 'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']

# topics 변수로 기재된 카테고리만 추출 + 머릿말, 꼬릿말, 각주 제거(remove)
news_df = fetch_20newsgroups(subset = 'all', remove=('headers', 'footers', 'quotes'), categories=topics, random_state = 0)

# Count 기반 백터화 변환
count_vect = CountVectorizer(max_df = 0.95, max_features = 1000, min_df=2, stop_words='english', ngram_range=(1,2))

In [4]:
feat_vect = count_vect.fit_transform(news_df.data)
feat_vect

<7862x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 207349 stored elements in Compressed Sparse Row format>

## LDA 토픽 모델링 수행

In [5]:
LDA = LatentDirichletAllocation(n_components=8, random_state=0)  # 토픽 개수를 8개로 조정
LDA.fit(feat_vect)

LatentDirichletAllocation(n_components=8, random_state=0)

In [7]:
print(LDA.components_.shape)   # components_의 shape 확인
LDA.components_    # components_의 속성값 확인

(8, 1000)


array([[3.60992018e+01, 1.35626798e+02, 2.15751867e+01, ...,
        3.02911688e+01, 8.66830093e+01, 6.79285199e+01],
       [1.25199920e-01, 1.44401815e+01, 1.25045596e-01, ...,
        1.81506995e+02, 1.25097844e-01, 9.39593286e+01],
       [3.34762663e+02, 1.25176265e-01, 1.46743299e+02, ...,
        1.25105772e-01, 3.63689741e+01, 1.25025218e-01],
       ...,
       [3.60204965e+01, 2.08640688e+01, 4.29606813e+00, ...,
        1.45056650e+01, 8.33854413e+00, 1.55690009e+01],
       [1.25128711e-01, 1.25247756e-01, 1.25005143e-01, ...,
        9.17278769e+01, 1.25177668e-01, 3.74575887e+01],
       [5.49258690e+01, 4.47009532e+00, 9.88524814e+00, ...,
        4.87048440e+01, 1.25034678e-01, 1.25074632e-01]])

In [8]:
#  각 토픽별로 연관도가 높은 순으로 단어를 나열하는 함수 생성
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        topic_word_indexes = topic.argsort()[::-1] #components_ 배열에서 가장 값이 큰 순으로 정렬했을 때
        top_indexes = topic_word_indexes[:no_top_words] # 그 값의 배열 인덱스를 반환

        feature_concat = ' '.join([feature_names[i] for i in top_indexes]) #top_indexes 대상인 인덱스 별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        print(feature_concat)

feature_names = count_vect.get_feature_names() #CountVectorizer 객체 내 전체 word 명칭을 get_features_names()를 통해 추출

C:\ProgramData\Anaconda3\envs\data\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
# 토픽별로 가장 연관도가 높은 단어 10개씩 추출
display_topics(LDA, feature_names, 10)

Topic # 0
year 10 game medical health team 12 20 disease cancer
Topic # 1
don just like know people said think time ve didn
Topic # 2
image file jpeg program gif images output format files color
Topic # 3
like know don think use does just good time book
Topic # 4
armenian israel armenians jews turkish people israeli jewish government war
Topic # 5
edu com available graphics ftp data pub motif mail widget
Topic # 6
god people jesus church believe christ does christian say think
Topic # 7
use dos thanks windows using window does display help like
